### Binary Tree

- A binary max-heap is a binary tree (each node has 0,1, or 2 children) where the value of each node is at least the values of its children
    - Min heap is the same definition, but with each node having at most the values of its children

- MaxHeap API
    - `GetMax`: O(1)
        - Returns the root value
    - `SiftUp`: O(log(N))
        - For a given node, check if it obeys the MaxHeap property
        - If it does not (i.e. child larger than parent), then swap it with parent
    - `SiftDown`: O(log(N))
        - For a given node, check if it obeys the MaxHeap property
        - If it does not (i.e. parent smaller than child), then check left and right child, and swap with the larger of the two
    - `Insert`: O(log(N))
        - Place the new node as the last leaf 
        - Then do `SiftUp` until the heap property is satisfied
    - `ExtractMax`: O(log(N))
        - Replace the root with the last leaf
        - Remove the last leaf from the heap, this is your max
        - `Siftdown` the root
    - `ChangePriority`: O(log(N))
        - If the new priority is higher, you want to sift up
        - If the new priority is lower, you want to sift down
    - `Remove`: O(log(N))
        - Replace the priority value to inf
        - Siftup
        - Call `ExtractMax`
    

- Complete binary tree
    - A tree is complete if all levels are filled except the last level, which is filled from left to right

- What are the advantages of a complete binary tree
    1. Short tree: A complete binary tree with $n$ nodes has at most O(log(N)) height. This means you can store a lot of nodes, but still get the max value in log(N) time!
    2. Store as array: You can represent the MaxHeap quite easily as an array
        - Assuming parent is index 1, a given node will have parent at node floor(i/2)
        - A given node will have children at 2i and 2i+1

- But what are our downsides?
    - We need to keep the tree complete for every operation. That is, anytime an operation modifies the shape of the tree, we need to ensure that the tree becomes complete again
        - Not every operation will involve changing the tree shape, only `Insert` and `ExtractMax`
    - `Insert` in the last vacant position and sift up, so insert takes log(N) time
    - `ExtractMax` swaps the root node (the max node) with the last leaf, and then sifts down, incurring log(N) time again

### Implementation

In [ ]:
# 0
# 1               2        
# 3       4       5       6
# 7   8   9   10  11  12  13  14

In [172]:
import math

class MaxHeap:
    def __init__(self, max_size, nodes):
        self.max_size=max_size
        self.nodes=nodes
        self.size=len(nodes)
        self.__post_init__()

    def __post_init__(self):
        if self.size != 0:
            self.heapify()
    
    def print_nodes(self):
        display(self.nodes)

    def parent(self, index, zero_index=True):
        # Assume 0 indexed
        if zero_index:
            return (index-1)//2
        else:
            return index//2

    def left_child(self, index, zero_index=True):
        if zero_index:
            return (2*index)+1
        else:
            return 2*index
    
    def right_child(self, index, zero_index=True):
        if zero_index:
            return (2*index)+2
        else:
            return (2*index)+1

    def sift_up(self, index, zero_index=True):
        if zero_index:
            min_index=0
        else:
            min_index=1
        
        curr_index=index
        while (curr_index > min_index) and (self.nodes[self.parent(curr_index)] < self.nodes[curr_index]):
            self.nodes[self.parent(curr_index)], self.nodes[curr_index] = self.nodes[curr_index], self.nodes[self.parent(curr_index)]

            curr_index = self.parent(curr_index)

    def sift_down(self, index, zero_index=True):
        if zero_index:
            min_index=0
        else:
            min_index=1
        
        left_child_index, right_child_index = self.left_child(index), self.right_child(index)
        left_child_value, right_child_value = -math.inf, -math.inf
        
        if (left_child_index >= self.size) and (right_child_index >= self.size):
            return
        
        if left_child_index < self.size:
            left_child_value = self.nodes[left_child_index]
        if right_child_index < self.size:
            right_child_value = self.nodes[right_child_index]
        
        curr_value = self.nodes[index]
        if left_child_value > right_child_value:
            swap_index = left_child_index
        else:
            swap_index = right_child_index

        if curr_value < max(curr_value, left_child_value, right_child_value):
            self.nodes[swap_index], self.nodes[index] = self.nodes[index], self.nodes[swap_index]

            self.sift_down(swap_index)

    def insert(self, value):
        if self.size == self.max_size:
            raise ValueError("Heap is max, unable to insert")
        else:
            self.size += 1
            self.nodes.append(value)
            self.sift_up(self.size-1)

    def extract_max(self):
        self.nodes[0], self.nodes[-1] = self.nodes[-1], self.nodes[0]
        maxval = self.nodes.pop()
        self.size -= 1
        self.sift_down(0)
        return maxval
    
    def remove(self, index):
        self.nodes[index] = math.inf
        self.sift_up(index)
        # print(self.nodes)

        self.extract_max()

    def change_priority(self, index, new_priority):
        previous_prior = self.nodes[index]
        self.nodes[index] = new_priority
        
        if new_priority > old_priority:
            self.sift_up(index)
        else:
            self.sift_down(index)

# mh = MaxHeap(max_size=100, nodes=[])
# mh.insert(0)
# mh.insert(1)
# mh.insert(2)
# mh.insert(3)
# mh.remove(2)

mh = MaxHeap(max_size=100, nodes=[0,1,2,3,4,5])

### Heap Sort

- For a given array, we can sort by building a heap, then iteratively doing `ExtractMax`! This is known as heapsort
    - Time complexity is O(N log(N))
        a. Turning an array into a heap takes O(N) time
        b. After it is turned into a heap, we call `ExtractMax` N times, with each call taking N log N
    - As such, overall time complexity is N log N

- We should pay some attention to point A above. How is it possible that turning an array into a heap (**heapify**) only takes O(N) time? Let's think through
    - There are 2 ways we can turn an array into a heap. Start from the bottom, `SiftDown`
    - Start from the top, and keep calling `SiftUp`

    - In the first case, we don't need to call `SiftDown` on the bottom row, because there is nothing below that layer.
        - This removes $\frac{n}{2}$ nodes from the loop
    - In the second case, we don't need to call `SiftUp` on the top row, because there is nothing above that layer.
        - This removes $1$ nodes from the loop

    - So it is clear that `SiftDown` is faster!!

- We now know `SiftDown` is faster. How many operations does the `SiftDown` actually take?
    - The bottom layer is left alone, so each of the $\frac{n}{2}$ nodes in bottom layer takes 0 operations
    - $\frac{n}{4}$ nodes in second last layer takes 1 operation
    - $\frac{n}{8}$ nodes in second last layer takes 2 operations
    - ...
    - 1 node takes $\log(N)$ operations

$$\begin{aligned}
\frac{n}{4} \cdot 1 + \frac{n}{8} \cdot 2 + ... 1 \cdot \log_2(n) &= \sum_{i=1}^{\log_2(n)} \frac{i \cdot n}{2^{i+1}} \\
&= \frac{n}{4} \sum_{i=1}^{\log_2(n)} \frac{i}{2^{i-1}} \\
&\lt \frac{n}{4} \sum_{i=1}^{\inf} \frac{i}{2^{i-1}} \\
&= \frac{n}{4} \sum_{i=1}^{\inf} i x^{i-1} & x = \frac{1}{2} \\
&= \frac{n}{4} \sum_{i=1}^{\inf} \frac{d}{dx} x^i \\
&= \frac{n}{4} \frac{d}{dx} \sum_{i=1}^{\inf} x^i \\
&= \frac{n}{4} \frac{d}{dx} \frac{1}{1-x} \\
&= \frac{n}{4} \frac{1}{(1-x)^2} \\
&= \frac{n}{4} \frac{1}{\frac{1}{4}} & x=\frac{1}{2} \\
&= n
\end{aligned}$$

- Thus, building a heap is O(N)!

In [178]:
import numpy as np
import types

input_array = list(np.random.randint(0, 100, 5))

def heapify(self):
    for i in range((self.size-2)//2, -1 , -1):
        self.sift_down(i)

MaxHeap.heapify = heapify

def heap_sort(self):

    self.heapify() #should already be a heap, O(N) otherwise
    output_sorted = []

    # extract max implicitly calls sift down, so it is O(log(N))
    # We do this for N nodes, giving us N log(N) time complexity
    while self.nodes:
        output_sorted.append(self.extract_max())
    return output_sorted
    
MaxHeap.heap_sort = heap_sort


mh = MaxHeap(max_size=100, nodes=input_array)
mh.heap_sort()

[99, 81, 71, 70, 62]

### Partial sorting

- Interestingly, the math above can also be used to show that while a **full sort** is $O(N \log(N))$, a **partial sort** may not be!

- Imagine we have an array $A[1,...N]$, and we want to output the last $k$ elements in the sorted version of $A$. Using the same approach as above
    - We first heapify the array $A$ in $O(N)$ time
    - Then we loop from 1 to $k$ and `ExtractMax`, taking $k \log(N)$ time
    - Overall time complexity is $O(N + K \log(N))$ 
        - But imagine if $k$ is sufficiently small such that $k \le \frac{N}{\log(N)}$
        - Then the time complexity simply becomes $O(N)$!

### d-ary heap

- As opposed to a binary heap, with every node has at most 2 children, it is also possible to construct a d-ary heap, where every node has at most $d$ children
    - In such a case, the height of the tree is $log_d(n)$ (as opposed to $\log_2(n)$ in the binary heap case)
    - Similarly, running `SiftUp` is now $O(\log_d(n))$ and `SiftDown` is $O(d \log_d(n))$